# 读入库

In [1]:
import jiagu
import string
import pandas as pd
import torch
from PIL import Image
import numpy as  np
import cn_clip.clip as clip
from cn_clip.clip import load_from_name, available_models
import itertools
import jiagu
import string
import re
import os
import fitz # PyMuPDF
import pdb
import ast
import math

from tqdm import tqdm

print("Available models:", available_models())
# Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']

/Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']


# 读入文本和图片信息的DataFrame

In [2]:
# 读入文本DataFrame
df_text = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/01 text/text_df_3.xlsx")
# df_text = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/partly/文本提取_text_df_2.xlsx")

# 读入图片DataFrame
df_img = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/03 PyMu_img01_coordinate/cord_final2.xlsx")

# 函数定义
## 去除字段中的标点和空格

In [3]:
def remove_punctuation_and_spaces(text):
    # 去除标点符号
    text = str(text)
    text = text.translate(str.maketrans("", "", string.punctuation))

    # 去除空格
    text = re.sub(r"\s+", "", text)

    return text

## 从文本段中提取关键词

In [4]:
# fin = open('GS.txt', mode='r',encoding='UTF-8')
# text = fin.read()
# fin.close()

def labelization(text, key_num = 3):
#     summarize = jiagu.summarize(text, 1) # 摘要
#     print("Sum:", summarize)
#     keywords = jiagu.keywords(text, 15) # 关键词
#     print("\nKeyWords:", keywords)

    cleaned_sentence = remove_punctuation_and_spaces(text)
#     print("\nCleaned Sentence:", cleaned_sentence)
    # summarize1 = jiagu.summarize(cleaned_sentence, 1) # 摘要
    # print(summarize1)
    keywords1 = jiagu.keywords(cleaned_sentence, key_num) # 关键词
#     print("KeyWords:", keywords1,"\n\n")
    
    return keywords1
# fin = open('input.txt', 'r')
# text = fin.read()
# fin.close()
#
# summarize = jiagu.summarize(text, 3) # 摘要
# print(summarize)

## 提取当页全部关键词

In [5]:
def extract_keywords(PDF_name, page):
    keywords_list = []
    keyword = []
#     page = page + 1
    
#     unique_pages = df_text[df_text["PDF_name"] == PDF_name]['page'].unique()
#     for page in unique_pages:
#         # 指定要获取的行
    page_rows = df_text[(df_text["PDF_name"] == PDF_name) & (df_text['page'] == page)]


    for idx, row in page_rows.iterrows():
        # 提取关键词
        if not pd.isna(row['content']):
            keywords = labelization(row['content'], len(str(row['content'])) // 20 + 2)
#         print(keywords)
        # 将关键词保存至特定行的'keyword'列
        df_text['keyword'] = df_text['keyword'].astype('object')
#         df_text.loc[idx, "keyword"] = keywords
        df_text.at[idx, 'keyword'] = keywords

        # 加入当前页的总'keyword_list'
        keywords_list.append(keywords)
        keyword = list(itertools.chain.from_iterable(keywords_list))
    
    return keyword

## 获取特定图片在当页标签中的概率

In [6]:
# img_path = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf_page_10_img_1.png"

# keywords = [keyword for keyword in keywords_list if pattern.match(keyword)]

def jiagu_prob(img_path, keywords):
    # 预处理，读入模型
    device='cpu'
    model, preprocess = load_from_name("RN50", device=device, download_root='./')
    model.eval()

    # 转移到mps上进行gpu加速
    device = "mps" if torch.backends.mps.is_available() == True else 'cpu'
    model = model.to(device)

    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    label = clip.tokenize(keywords).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(label)
        # 对特征进行归一化，用于下游任务
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        logits_per_image, logits_per_text = model.get_similarity(image, label)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    return probs, img_path

# print("\nLabel probs:", probs)  # [[1.268734e-03 5.436878e-02 6.795761e-04 9.436829e-01]]
# labels = keywords
# max_prob_index = np.argmax(probs)
# max_prob_label = labels[max_prob_index]
# print("\nMax probability label:", max_prob_label)

## 图片匹配文字

In [7]:
def text_match_img(labels, probs, img_path, PDF_name, page, df_text = df_text, df_img = df_img):
    """
    将每个图片最配对文本保留在df_img中
    将最配对文本的可信度（CRI）保留
    """
    # 创建一个标签到概率的映射字典
    prob_dict = dict(zip(labels, probs[0]))
#     print("\nProb_Dict:\n", prob_dict, "\n\n\n")
    filtered_df = df_text[(df_text["PDF_name"] == PDF_name) & (df_text['page'] == page)]
    if not filtered_df.empty:
        filtered_df = filtered_df[pd.notnull(filtered_df['keyword'])]

        # 创建一个新列来存储每个标签组的概率总和
        filtered_df['prob_sum'] = filtered_df['keyword'].apply(
            lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) / 
                             (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
        )

        
        # 使用 apply() 方法遍历列
        filtered_df['prob_sum'] = filtered_df['prob_sum'].apply(lambda x: 0.0 if not isinstance(x, float) else x)
#         print("\nFiltered_df:\n",filtered_df, "\n")
        
        if not filtered_df['prob_sum'].empty:
            max_prob_content = filtered_df.loc[filtered_df['prob_sum'].idxmax(), 'content']
            max_prob = filtered_df['prob_sum'].max()
            cri = CRI(filtered_df)
        else:
        # 如果 prob_sum 列为空
            max_prob_content = None  # 或任何合适的默认值
            max_prob = None
            cri = None

        # 获取图片名称并存储至df_img
        img_name = os.path.basename(img_path)
        df_img.loc[df_img["file_name"] == img_name, "match_text"] = max_prob_content
        df_img.loc[df_img["file_name"] == img_name, "match_text_prob"] = max_prob
        df_img.loc[df_img["file_name"] == img_name, "match_text_CRI"] = cri
        
        return max_prob_content

## 计算标签组概率的可靠性

In [8]:
def CRI(filtered_df):
    """
    综合可靠性指数(Composite Reliability Index, CRI)
        计算: 最大概率差异度(Maximum Probability Spread, MPS) / 标准差(Standard Deviation, SD)
    
    最大概率差异度(Maximum Probability Spread, MPS): 
        计算: 最大概率值与其他概率值的差值，然后取平均值。
    """
    # 计算MPS和SD
    max_probability = filtered_df['prob_sum'].max()  # 最大概率值
    average_probability = filtered_df['prob_sum'].mean()  # 所有概率的平均值
    probabilities_except_max = filtered_df[filtered_df['prob_sum'] != max_probability]['prob_sum']  # 除了最大值的所有概率值

    # 计算MPS
    mps = max_probability - probabilities_except_max.mean() if not probabilities_except_max.empty else 0

    # 计算标准差
    sd = filtered_df['prob_sum'].std()  # 所有概率值的标准差

    # 计算CRI
    cri = mps / sd if sd > 0 else 0  # 如果标准差为0，则CRI没有意义
    
    return cri

## 一次获取PDF中每一页的全部标签

In [9]:
def label_in_PDF(PDF_path):
    pdf_file = fitz.open(PDF_path)
    pdf_name = os.path.basename(PDF_path)

    key = []

    # 遍历PDF的每一页
    for page in tqdm(range(len(pdf_file))):
        keywords = extract_keywords(pdf_name, page)
    #     print("Page:{}\n{}\n\n".format(page,keywords))
        key.append({'file_name': pdf_name,
                    'page': page + 1,
                    'keywords': keywords})

    # 创建临时df存放每一页的关键词组 
    df_key = pd.DataFrame(key)
    
    return df_key

## 计算距离

In [10]:
def distance(img_cord, text_cord):
    """
    Takes in both 'img_cord' and 'text_cord' in tuple of (x, y).
    Return the distance between two cord.
    """
    img_x, img_y = img_cord
    text_x, text_y = text_cord
    
    return math.sqrt((text_x - img_x)**2 + (text_y - img_y)**2)

In [11]:
def get_bundle_path(directory):
    files = [f for f in os.listdir(directory) if f != '.DS_Store']
    return [os.path.join(directory, file) for file in files if os.path.isfile(os.path.join(directory, file))]

# 计算最近的图文间的语义匹配度

In [12]:
def wording_similarity(labels, probs, img_path, PDF_name, page, df_text = df_text, df_img = df_img):
    # 创建一个标签到概率的映射字典
    prob_dict = dict(zip(labels, probs[0]))
    
    # 获得图片坐标
    img_name = os.path.basename(img_path)
    img_cord = df_img[df_img['file_name'] == img_name]['centre_coordinate']
    [img_cord] = img_cord.values
    img_cord = ast.literal_eval(img_cord)
        
    # 获取文本段坐标
    df_min_dis = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page)]
#     df_min_dis = df_min_dis(pd.notnull(df_min_dis['keyword']))
    if not df_min_dis.empty:
        df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)
    else:
        print("{0:=^40}".format('Anomaly Founded'))
        print("PDF:{} has no text content on page{}".format(PDF_name, page))
    
    # 计算标签组概率
    return df_min_dis.sort_values(by='dis_to_img_current', ascending = True)[0:1]['keyword'].apply(
        lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) /
                         (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
    )

In [ ]:
img_path = '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf_page_7_img_1.png'
PDF_path = '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf'
df_key = label_in_PDF(PDF_path)
[keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == 8)]['keywords'])

probs, img_path = jiagu_prob(img_path, keywords)

wording_similarity(keywords, probs, img_path, os.path.basename(PDF_path), page=7)

# 运行

In [ ]:
# 指定要处理的PDF路径
# PDF_path = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf"

In [13]:
# 指定要处理的PDF文件夹
# folder_dir = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022"
folder_dir = "/Users/improvise/Downloads/test"

# 指定要处理的图片文件夹
img_folder = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01/"

In [14]:
PDF_path_list = get_bundle_path(folder_dir)

for PDF_path in PDF_path_list:
    df_key = label_in_PDF(PDF_path)
    for row in df_img.loc[df_img['PDF_name']==os.path.basename(PDF_path)].itertuples():
        img_name = row.file_name
        page = row.page
        p_index = row.p_index
        img_cord = row.centre_coordinate

        # 读图
        img_path = os.path.join(img_folder, img_name)
        # 读页面关键词
        [keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == page)]['keywords'])
        # 计算概率
        probs, img_path = jiagu_prob(img_path, keywords)

        # 匹配图文
        object_text = text_match_img(keywords, probs, img_path, os.path.basename(PDF_path), page)
        if object_text != None:
        # 获取文本坐标
            text_cord_list = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page) & (df_text['content'] == object_text)][['center_x','center_y']].values.tolist()

            if text_cord_list:
                text_cord = text_cord_list[0]  # 选择第一个元素

            # 接下来的计算
            else:
                [text_cord] = text_cord_list

            # 计算距离
            if type(img_cord) == str:
                    img_cord = ast.literal_eval(img_cord)

            dist = distance(img_cord, text_cord)
            df_img.loc[df_img["file_name"] == img_name, "distance"] = dist

100%|███████████████████████████████████████████| 50/50 [00:02<00:00, 20.14it/s]


Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/minicon

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/minicon

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/minicon

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/minicon

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/minicon

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/minicon

Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/minicon

In [ ]:
# for row in df_img.loc[df_img['PDF_name']==os.path.basename(PDF_path)].itertuples():
#     img_name = row.file_name
#     page = row.page
#     p_index = row.p_index
#     img_cord = row.centre_coordinate
    
#     # 读图
#     img_path = os.path.join(img_folder, img_name)
#     # 读页面关键词
#     [keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == page)]['keywords'])
#     # 计算概率
#     probs, img_path = jiagu_prob(img_path, keywords)
    
#     # 匹配图文
#     object_text = text_match_img(keywords, probs, img_path, os.path.basename(PDF_path), page)
#     if object_text != None:
#     # 获取文本坐标
#         text_cord_list = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page) & (df_text['content'] == object_text)][['center_x','center_y']].values.tolist()

#         if text_cord_list:
#             text_cord = text_cord_list[0]  # 选择第一个元素
            
#         # 接下来的计算
#         else:
#             [text_cord] = text_cord_list
        
#         # 计算距离
#         if type(img_cord) == str:
#                 img_cord = ast.literal_eval(img_cord)

#         dist = distance(img_cord, text_cord)
#         df_img.loc[df_img["file_name"] == img_name, "distance"] = dist

In [16]:
df_img.to_excel("/Users/improvise/Downloads/distance.xlsx", index=False)